## Denovo M7 FLYE assemblies using different subsets of data

In [2]:
WKDIR=/workspace/hraijc/Bee/
ONTREADS=/workspace/hraijc/Bee/input/Leioproctus_imitatus/ONT/Bee_ONT_M1F2.fastq
ILLREADS=/workspace/hraijc/Bee/input/Leioproctus_imitatus/popgen/

In [2]:
mkdir $WKDIR/log

In [3]:
module load conda
conda activate hraijc_flye
python /workspace/hraijc/git_clones/Flye/bin/flye -v

In [4]:
python /workspace/hraijc/git_clones/Flye/bin/flye -v

2.9.2-b1794


In [5]:
python /workspace/hraijc/git_clones/Flye/bin/flye -h

usage: flye (--pacbio-raw | --pacbio-corr | --pacbio-hifi | --nano-raw |
	     --nano-corr | --nano-hq ) file1 [file_2 ...]
	     --out-dir PATH

	     [--genome-size SIZE] [--threads int] [--iterations int]
	     [--meta] [--polish-target] [--min-overlap SIZE]
	     [--keep-haplotypes] [--debug] [--version] [--help] 
	     [--scaffold] [--resume] [--resume-from] [--stop-after] 
	     [--read-error float] [--extra-params] 
	     [--deterministic]

Assembly of long reads with repeat graphs

optional arguments:
  -h, --help            show this help message and exit
  --pacbio-raw path [path ...]
                        PacBio regular CLR reads (<20% error)
  --pacbio-corr path [path ...]
                        PacBio reads that were corrected with other methods
                        (<3% error)
  --pacbio-hifi path [path ...]
                        PacBio HiFi reads (<1% error)
  --nano-raw path [path ...]
                        ONT regular reads, pre-Guppy5 (<20% error)
  --nano-c

In [5]:
##FLYE MANUAL:
#For the most recent ONT data basecalled with Guppy5+ SUP use the new --nano-hq mode. Expected error rate is <5%.
#For Q20 data, use a combination of --nano-hq and --read-error 0.03.

(/workspace/appscratch/miniconda/hraijc_flye) (/workspace/appscratch/miniconda/hraijc_flye) (/workspace/appscratch/miniconda/hraijc_flye) 

: 1

In [16]:
#module load conda
#conda activate hraijc_flye

#Assemble ONT test runs. ~5x coverage
sbatch << EOF
#!/bin/bash
#SBATCH -J Flye
#SBATCH --cpus-per-task=16
#SBATCH --mem=20G
#SBATCH --time=12:00:00
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err


cd $WKDIR



python /workspace/hraijc/git_clones/Flye/bin/flye --nano-raw /workspace/hraijc/Bee/input/Leioproctus_imitatus/ONT/Bee_ONT_M1F2.fastq --out-dir /workspace/hraijc/Bee/ONT_Test1 --genome-size 200m --threads 16 --iterations 2


EOF

Submitted batch job 1360231


In [9]:
module unload conda

(/workspace/appscratch/miniconda/hraijc_flye) 

: 1

### Assemble with Illumina Data from PopGen work.

In [2]:
FLYE_T1=/workspace/hraijc/Bee/ONT_Test1/assembly.fasta
I1=${ILLREADS}/NI-FR-Li-F-61-40567916_S16_L004_R1_001.clean.fastq.gz 
I2=${ILLREADS}/NI-FR-Li-F-61-40567916_S16_L004_R2_001.clean.fastq.gz
WKDIR=/workspace/hraijc/Bee/
ONTREADS=/workspace/hraijc/Bee/input/Leioproctus_imitatus/ONT/Bee_ONT_M1F2.fastq


In [2]:
module load fastp

In [18]:

#CleanReads
sbatch << EOF
#!/bin/bash
#SBATCH -J fastp
#SBATCH --cpus-per-task=1
#SBATCH --mem=2G
#SBATCH --time=2:00:00
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err


cd $WKDIR
module load fastp


fastp -i ${ILLREADS}/NI-FR-Li-F-61-40567916_S16_L004_R1_001.fastq.gz -I ${ILLREADS}/NI-FR-Li-F-61-40567916_S16_L004_R2_001.fastq.gz -o ${ILLREADS}/NI-FR-Li-F-61-40567916_S16_L004_R1_001.clean.fastq.gz -O ${ILLREADS}/NI-FR-Li-F-61-40567916_S16_L004_R2_001.clean.fastq.gz


EOF

Submitted batch job 1360263


In [3]:
#### SPADES
module load SPAdes/3.15.4 

In [32]:
#spades.py -h

In [6]:
#Spades on Illumina data only
sbatch << EOF
#!/bin/bash
#SBATCH -J Spades
#SBATCH --cpus-per-task=16
#SBATCH --mem=20G
#SBATCH --time=12:00:00
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err


cd $WKDIR

module load SPAdes/3.15.4 

#spades.py -k 21,33,55,77 --careful -1 ${I1} -2 ${I2} -o ${WKDIR}/spades_output
spades.py --continue -o ${WKDIR}/spades_output


EOF



Submitted batch job 1360751


In [3]:
#Spades on Illumina data and ONT Data
sbatch << EOF
#!/bin/bash
#SBATCH -J Spades
#SBATCH --cpus-per-task=16
#SBATCH --mem=20G
#SBATCH --time=22:00:00
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err


cd $WKDIR

module load SPAdes/3.15.4 

#spades.py -k 21,33,55,77 --continue --careful -1 ${I1} -2 ${I2} --nanopore ${ONTREADS} -o ${WKDIR}/spades_output_ONT
spades.py --continue -o ${WKDIR}/spades_output_ONT

EOF



Submitted batch job 1403001


In [1]:
seff 1360752


Job ID: 1360752
Cluster: powerplant
User/Group: hraijc/hraijc
State: TIMEOUT (exit code 0)
Nodes: 1
Cores per node: 16
CPU Utilized: 01:21:34
CPU Efficiency: 0.71% of 8-00:04:32 core-walltime
Job Wall-clock time: 12:00:17
Memory Utilized: 3.07 GB
Memory Efficiency: 15.34% of 20.00 GB


In [27]:
module load BBMap/38.33
statswrapper.sh in=$FLYE_T1 format=3
module unload BBMap/38.33

java -ea -Xmx200m -cp /software/bioinformatics/BBMap-38.33/current/ jgi.AssemblyStatsWrapper format=3 in=/workspace/hraijc/Bee/ONT_Test1/assembly.fasta format=3
n_scaffolds	n_contigs	scaf_bp	contig_bp	gap_pct	scaf_N50	scaf_L50	ctg_N50	ctg_L50	scaf_N90	scaf_L90	ctg_N90	ctg_L90	scaf_max	ctg_max	scaf_n_gt50K	scaf_pct_gt50K	gc_avg	gc_std	filename
593	593	5223673	5223673	0.000	131	12626	131	12626	375	4855	375	4855	59836	59836	1	1.145	0.36344	0.05878	/powerplant/workspace/hraijc/Bee/ONT_Test1/assembly.fasta


In [29]:
#module load conda
#conda activate hraijc_flye

#Assemble ONT test runs. ~5x coverage
sbatch << EOF
#!/bin/bash
#SBATCH -J Busco
#SBATCH --cpus-per-task=12
#SBATCH --mem=2G
#SBATCH --time=2:00:00
#SBATCH -o ${WKDIR}/log/%J.out
#SBATCH -e ${WKDIR}/log/%J.err


cd $WKDIR
module load BUSCO/v5.2.2


busco -i $FLYE_T1 -l insecta -o Bee_ONT_Test1 -m geno -c 12 --datasets_version odb10


EOF

Submitted batch job 1360293
